In [ ]:
from xtquant.xttrader import XtQuantTrader, XtQuantTraderCallback
from xtquant.xttype import StockAccount
import random

error_orders = []

class MyXtQuantTraderCallback(XtQuantTraderCallback):
    def on_disconnected(self):
        """
        连接状态回调
        :return:
        """
        print("connection lost")
    def on_stock_order(self, order):
        """
        委托信息推送
        :param order: XtOrder对象
        :return:
        """
        # 委托
        if order.order_status == 50:
            print(f"【已委托】 {order.order_type} 代码:{order.stock_code} 名称:{order.order_remark} 委托价格:{order.price:.2f} 委托数量:{order.order_volume} 订单编号:{order.order_id} 委托时间:{order.order_time}")
        elif order.order_status == 53 or order.order_status == 54:
            print(f"【已撤单】{order.order_type} 代码:{order.stock_code} 名称:{order.order_remark} 委托价格:{order.price:.2f} 委托数量:{order.order_volume} 订单编号:{order.order_id} 委托时间:{order.order_time}")

    def on_stock_trade(self, trade):
        """
        成交信息推送
        :param trade: XtTrade对象
        :return:
        """
        print(f"【已成交】{trade.order_type} 代码:{trade.stock_code} 名称:{trade.order_remark} 成交价格:{trade.traded_price:.2f} 成交数量:{trade.traded_volume} 成交编号:{trade.order_id} 成交时间:{trade.traded_time}")

    def on_order_error(self, data):
        if data.order_id in error_orders:
            return
        error_orders.append(data.order_id)
        print(f"【委托失败】错误信息:{data.error_msg.strip()}")

    def on_cancel_error(self, data):
        if data.order_id in error_orders:
            return
        error_orders.append(data.order_id)
        print(f"【撤单失败】错误信息:{data.error_msg.strip()}")


def create_trader(account_id, mini_qmt_path):
    # 创建session_id
    session_id = int(random.randint(100000, 999999))
    # 创建交易对象
    xt_trader = XtQuantTrader(mini_qmt_path, session_id)
    # 启动交易对象
    xt_trader.start()
    # 连接客户端
    connect_result = xt_trader.connect()

    if connect_result == 0:
        print(f"【miniQMT连接成功】路径:{mini_qmt_path}")

    # 创建账号对象
    account = StockAccount(account_id)
    # 订阅账号
    xt_trader.subscribe(account)
    print(f"【账号订阅成功】账号ID:{account_id}")
    # 注册回调类
    xt_trader.register_callback(MyXtQuantTraderCallback())

    return xt_trader, account


In [ ]:
xt_trader, account = create_trader("55003152", r"D:\国金QMT交易端模拟\userdata_mini")

In [ ]:
from xtquant import xtconstant
order = xt_trader.order_stock(account, stock_code='600519.SH', order_type=xtconstant.STOCK_BUY, price_type=xtconstant.LATEST_PRICE, order_volume=100, price=-1)
print(order)

In [ ]:
from xtquant import xtconstant
res = xt_trader.order_stock_async(account, stock_code='600519.SH', order_type=xtconstant.STOCK_BUY, price_type=xtconstant.LATEST_PRICE, order_volume=100, price=-1)
print(res)

In [ ]:
orders = xt_trader.query_stock_orders(account, cancelable_only = True)
print(orders)
for order in orders:
    res = xt_trader.cancel_order_stock(account, order.order_id)
    print(res)